In [40]:
from zope.interface import Interface, implementer
from zope.interface.verify import verifyClass
import attr
import sys
import pyrsistent
from pyrsistent import m, v
from __future__ import annotations
import pprint
import textwrap

```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}

\title{Why Change?}
\subtitle{Life is Better Without It}
\author{Moshe Zadka -- https://cobordism.com}
\date{2020}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

```{=latex}

\begin{frame}
\frametitle{Shared Mutable State is Bad}

Options:

\begin{itemize}
\item Don't share
\item Don't mutate
\end{itemize}

\end{frame}
```


Shared mutable state is bad. Sometimes people associate that badness with threads, but threads are just one example of how shared mutable state is bad. In Python, it's not even *particularly* bad. Your dictionary will still stay a dictionary. You just might find that the value of a key is different than you expected it tor be.
But, after all, this can happen even if you sent this dictionary to a function. Or made the dictionary be a class or instance variable. Or had a function capture it via a closure, then send it to another object, which finally mutated it. 

If shared mutable state is bad, there are only two ways to avoid it. One way is to avoid sharing. This is *really* hard in Python. Objects are thrown around all the time. The only other way is to avoid mutating.

```{=latex}

\begin{frame}
\frametitle{Avoid Sharing?}

What about

\begin{itemize}
\item Modules
\item Function defaults
\item Class variables
\item Arguments
\end{itemize}

\end{frame}
```


But, after all, this can happen even if you sent this dictionary to a function. Or made the dictionary be a class or instance variable. Or had a function capture it via a closure, then send it to another object, which finally mutated it. 

```{=latex}

\begin{frame}
\frametitle{Avoid Mutating!}

Much better.

\end{frame}
```


If shared mutable state is bad, there are only two ways to avoid it. One way is to avoid sharing. This is *really* hard in Python. Objects are thrown around all the time. The only other way is to avoid mutating. In this talk, we will explore programs without change. Not only will this neatly avoid shared mutable change, I will explore other benefits from having constancy.

```{=latex}

\begin{frame}
\frametitle{Digression: Are Squares Rectangles?}

...and why would anyone care?

\end{frame}
```


```{=latex}
\begin{frame}[fragile]
\frametitle{What's a Rectangle (in Python)}
```

In [41]:
class IRectangle(Interface):
    def get_height() -> float:
        """Return height"""
    def get_width() -> float:
        """Return width"""
    def set_height(height: float):
        """Set height"""
    def set_width(width: float):
        """Set width"""

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{What's a Square (in Python)}
```

In [56]:
@implementer(IRectangle)
@attr.s(auto_attribs=True)
class Square:
    _side: float
    def get_height(self) -> float: return self._side
    def get_width(self) -> float: return self._side
try: verifyClass(IRectangle, Square)
except Exception as exc:
    print(textwrap.dedent(str(exc).split(":")[1]
          ).strip().replace("__main__.", ""),
          file=sys.stderr)

The IRectangle.set_height(height) attribute was not provided
The IRectangle.set_width(width) attribute was not provided


```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{What's a Square (in Python) (Fixed)}

An easy mistake -- we forgot a couple of methods.

Let's fix that.
```

In [58]:
@implementer(IRectangle)
@attr.s(auto_attribs=True)
class Square:
    _side: float
    def get_height(self) -> float: return self._side
    def get_width(self) -> float: return self._side
    def set_height(self, height: float):
        self._side = height # ???
    def set_width(self, width: float):
        self._side = width # ???

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{What Do You Do With a Shape (in Python)}
```

In [59]:
def area(rectangle: IRectangle) -> float:
    return (rectangle.get_height() *
            rectangle.get_width())
def double_height(rectangle: IRectangle) -> float:
    rectangle.set_height(
        2 * rectangle.get_height())
x = Square(side=5)
print(area(x))
double_height(x)
print(area(x))

25
100


```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Let's Stop Mutating}
```

In [64]:
class IRectangle(Interface):
    def get_height() -> float:
        """Return height"""
    def get_width() -> float:
        """Return width"""
    def with_height(height: float) -> IRectangle:
        """Rectangle with same width, new height"""
    def with_width(width: float) -> IRectangle:
        """Rectangle with same height, new width"""

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{The Immutable Rectangle}
```

In [65]:
@implementer(IRectangle)
@attr.s(auto_attribs=True, frozen=True)
class Rectangle:
    _height: float
    _width: float
    def get_height(self) -> float:
        return self._height 
    def get_width(self) -> float:
        return self._width
    def with_height(self, height) -> float:
        return attr.evolve(self, height=height)
    def with_width(self, width) -> float:
        return attr.evolve(self, width=width)

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{The Immutable Square}
```

In [66]:
@implementer(IRectangle)
@attr.s(auto_attribs=True)
class Square:
    _side: float
    def get_height(self) -> float:
        return self._side
    def get_width(self) -> float:
        return self._side
    def with_height(self, height: float) -> IRectangle:
        return Rectangle(width=self._side,
                         height=height)
    def with_width(self, width: float) -> IRectangle:
        return Rectangle(height=self._side,
                         width=width)
verifyClass(IRectangle, Square)

True

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{What Do You Do With an Immutable Shape (in Python)}
```

In [67]:
def double_height(rectangle):
    return rectangle.with_height(
        2 * rectangle.get_height())
x = Square(side=5)
print(area(x))
print(area(double_height(x)))

25
50


```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Let's Get Back to Sharing}

At some point, someone told you not to do this.
Do you remember why?
```

In [80]:
def sum_with_extra(e1, e2, things=[]):
    things.append(e1)
    things.append(e2)
    return sum(things)

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{A Bad Trip Down Memory Lane}
```

In [81]:
sum_with_extra(1, 2, [3, 4])

10

In [82]:
sum_with_extra(1, 2)

3

In [83]:
# Whoops!
sum_with_extra(1, 2)

6

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{The Fix is Easy!}
```

In [92]:
def sum_with_extra_v2(e1, e2, things=None):
    if things is None:
        things = []
    things.append(e1)
    things.append(e2)
    return sum(things)

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Everything is Awesome!}
```

In [93]:
sum_with_extra_v2(1, 2, [3, 4])

10

In [94]:
sum_with_extra_v2(1, 2)

3

In [95]:
sum_with_extra_v2(1, 2)

3

In [96]:
things = [1, 2]; sum_with_extra_v2(1, 2, things)

6

In [97]:
# Whoops!
sum_with_extra_v2(1, 2, things)

9

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{One Urgent Hot Fix Later...}

We got it to work!
```

In [26]:
def sum_with_extra_v3(e1, e2, things=None):
    if things is None:
        things = []
    things = things.copy()
    things.append(e1)
    things.append(e2)
    return sum(things)

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{..Meanwhile, Without Mutation}

Let's throw caution to the wind and live our best life.
```

In [98]:
def sum_with_extra_p_v1(e1, e2, things=v()):
    things = things.append(e1)
    things = things.append(e2)
    return sum(things)

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{We Don't Need v2}
```

In [99]:
sum_with_extra_p_v1(1, 2)

3

In [100]:
sum_with_extra_p_v1(1, 2)

3

In [101]:
things = v(1, 2)
sum_with_extra_p_v1(1, 2, things)

6

In [102]:
sum_with_extra_p_v1(1, 2, things)

6

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{But Nested Data Structures Are a Drag?}
```

How do you increase the hits on `web_1`?


In [34]:
stats = m(
    frontend=m(
        web_1=m(hits=53),
        web_2=m(hits=78)),
    backend=m(
        db1=m(queries=23),
        db2=m(queries=11)))

```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}[fragile]
\frametitle{Like This}
```

In [106]:
new_stats = stats.transform(
    v("frontend", "web_1", "hits"), 
    lambda x: x + 1)
pprint.pprint(pyrsistent.thaw(new_stats),
              width=50)

{'backend': {'db1': {'queries': 23},
             'db2': {'queries': 11}},
 'frontend': {'web_1': {'hits': 54},
              'web_2': {'hits': 78}}}


```{=latex}
\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Conclusion}

\begin{itemize}
\item Sharing good
\item Mutation bad
\item Share more
\item Mutate less
\item Be happy
\end{itemize}

\end{frame}
```

```{=latex}
\end{document}
```